In [1]:
# Pre-requisites
import numpy as np
import time

# For plots
%matplotlib inline
import matplotlib.pyplot as plt

# To clear print buffer
from IPython.display import clear_output

# Importing functions from the previous tutorials:

In [84]:
# Initializing weight matrices from layer sizes
def initializeWeights(layers):
    weights = [np.random.randn(o, i+1) for i, o in zip(layers[:-1], layers[1:])]
    return weights

# Add a bias term to every data point in the input
def addBiasTerms(X):
        # Make the input an np.array()
        X = np.array(X)
        
        # Forcing 1D vectors to be 2D matrices of 1xlength dimensions
        if X.ndim==1:
            X = np.reshape(X, (1, len(X)))
        
        # Inserting bias terms
        X = np.insert(X, 0, 1, axis=1)
        
        return X

# Sigmoid function
def sigmoid(a):
    return 1/(1 + np.exp(-a))

# Forward Propagation of outputs
def forwardProp(X, weights):
    # Initializing an empty list of outputs
    outputs = []
    
    # Assigning a name to reuse as inputs
    inputs = X
    
    # For each layer
    for w in weights:
        # Add bias term to input
        inputs = addBiasTerms(inputs)
        
        # Y = Sigmoid ( X .* W^T )
        outputs.append(sigmoid(np.dot(inputs, w.T)))
        
        # Input of next layer is output of this layer
        inputs = outputs[-1]
        
    return outputs

# Compute COST (J) of Neural Network
def nnCost(weights, X, Y):
    # Calculate yPred
    yPred = forwardProp(X, weights)[-1]
    
    # Compute J
    J = 0.5*np.sum((yPred-Y)**2)/len(Y)
    
    return J

# IMPLEMENTING BACK-PROPAGATION WITH LEARNING RATE
# Added eta, the learning rate, as an input
def backProp(weights, X, Y, learningRate):
    # Forward propagate to find outputs
    outputs = forwardProp(X, weights)
    
    # For the last layer, bpError = error = yPred - Y
    bpError = outputs[-1] - Y
    
    # Back-propagating from the last layer to the first
    for l, w in enumerate(reversed(weights)):
        
        # Find yPred for this layer
        yPred = outputs[-l-1]
        
        # Calculate delta for this layer using bpError from next layer
        delta = np.multiply(np.multiply(bpError, yPred), 1-yPred)
        
        # Find input to the layer, by adding bias to the output of the previous layer
        # Take care, l goes from 0 to 1, while the weights are in reverse order
        if l==len(weights)-1: # If 1st layer has been reached
            xL = addBiasTerms(X)
        else:
            xL = addBiasTerms(outputs[-l-2])
        
        # Calculate the gradient for this layer
        grad = np.dot(delta.T, xL)/len(Y)
        
        # Calculate bpError for previous layer to be back-propagated
        bpError = np.dot(delta, w)
        
        # Ignore bias term in bpError
        bpError = bpError[:,1:]
        
        # Change weights of the current layer (W <- W + eta*deltaW)
        w += -learningRate*grad

# Evaluate the accuracy of weights for input X and desired outptut Y
def evaluate(weights, X, Y):
    yPreds = forwardProp(X, weights)[-1]
    # Check if maximum probability is from that neuron corresponding to desired class,
    # AND check if that maximum probability is greater than 0.5
    yes = sum( int( ( np.argmax(yPreds[i]) == np.argmax(Y[i]) ) and 
                    ( (yPreds[i][np.argmax(yPreds[i])]>0.5) == (Y[i][np.argmax(Y[i])]>0.5) ) )
              for i in range(len(Y)) )
    return yes

# TRAINING USING MINI-BATCH GRADIENT DESCENT
# Default learning rate = 1.0
def trainUsingMinibatchGD(weights, X, Y, minibatchSize, nEpochs, learningRate=1.0, testX=None, testY=None):
    # If testing data is not provided, check accuracy on training data
    if testX is None:
        testX = X
        testY = Y
    
    # For nEpochs number of epochs:
    for i in range(nEpochs):
        # clear output
        #clear_output()
        
        # Make a list of all the indices
        fullIdx = list(range(len(Y)))
        
        # Shuffle the full index
        np.random.shuffle(fullIdx)
        
        # Count number of mini-batches
        nOfMinibatches = int(len(X)/minibatchSize)
        
        # For each mini-batch
        for m in range(nOfMinibatches):
            
            # Compute the starting index of this mini-batch
            startIdx = m*minibatchSize
            
            # Declare sampled inputs and outputs
            xSample = X[fullIdx[startIdx:startIdx+minibatchSize]]
            ySample = Y[fullIdx[startIdx:startIdx+minibatchSize]]
            
            # Run backprop
            backProp(weights, xSample, ySample, learningRate)
        
        # Check cost and accuracy
        cost = nnCost(weights, testX, testY)
        yes = evaluate(weights, testX, testY)
        print("Epoch "+str(i+1)+" of "+str(nEpochs)+" : "+
              str(yes)+" out of "+str(len(testY))+" = "+str(round(float(yes/len(testY)),4))+
              "; cost="+str(round(cost, 4)))

# Initialize network
layers = [2, 2, 1]
weights = initializeWeights(layers)

print("weights:")
for i in range(len(weights)):
    print(i+1); print(weights[i].shape); print(weights[i])

# Declare input and desired output for AND gate
X = np.array([[0,0], [0,1], [1,0], [1,1]])
Y = np.array([[0], [0], [0], [1]])

# Check current accuracy and cost
print("Cost: "+str(nnCost(weights, X, Y)))
print("Accuracy: ")
evaluate(weights, X, Y)
print(forwardProp(X, weights)[-1])

weights:
1
(2, 3)
[[-0.29285599 -1.17220739 -0.85551856]
 [-0.84896999 -1.66102533  0.15182479]]
2
(1, 3)
[[ 1.54946039 -0.34411433  0.87618229]]
Cost: 0.267971323763
Accuracy: 
[[ 0.84090348]
 [ 0.85294461]
 [ 0.82501862]
 [ 0.83123477]]


# Load MNIST data, and format it for our network

In [85]:
# Load MNIST DATA
# Use numpy.load() to load the .npz file
f = np.load('mnist.npz')
# Saving the files
x_train = f['x_train']
y_train = f['y_train']
x_test = f['x_test']
y_test = f['y_test']
f.close()

# Reshaping x_train and x_test for our network with 784 inputs neurons
x_train = np.reshape(x_train, (len(x_train), 784))
x_test = np.reshape(x_test, (len(x_test), 784))

# Normalize x_train
x_train = x_train / 255.0
x_test = x_test / 255.0

# Make new y_train of nx10 elements
new_y_train = np.zeros((len(y_train), 10))
for i in range(len(y_train)):
    new_y_train[i, y_train[i]] = 1

del y_train
y_train = new_y_train

# Make new y_test of nx10 elements
new_y_test = np.zeros((len(y_test), 10))
for i in range(len(y_test)):
    new_y_test[i, y_test[i]] = 1

del y_test
y_test = new_y_test


## Try to train our network to give good accuracy on MNIST data

##Let us try a deeper network, with 2 hidden layers of 10 neurons each. And let us test it on test data.

In [138]:
# TRAIN using Batch Gradient Descent

# Initialize network
layers = [784, 30, 10]
weights = initializeWeights(layers)

# Take backup of weights to be used later for comparison
initialWeights = [np.array(w) for w in weights]

# Set options of mini-batch gradient descent
minibatchSize = len(y_train)
nEpochs = 30
learningRate = 1.0

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate, testX=x_test, testY=y_test)

Epoch 1 of 30 : 1021 out of 10000 = 0.1021; cost=2.3735
Epoch 2 of 30 : 979 out of 10000 = 0.0979; cost=2.2225
Epoch 3 of 30 : 971 out of 10000 = 0.0971; cost=2.1205
Epoch 4 of 30 : 970 out of 10000 = 0.097; cost=2.0373
Epoch 5 of 30 : 968 out of 10000 = 0.0968; cost=1.9495
Epoch 6 of 30 : 964 out of 10000 = 0.0964; cost=1.8392
Epoch 7 of 30 : 976 out of 10000 = 0.0976; cost=1.6981
Epoch 8 of 30 : 968 out of 10000 = 0.0968; cost=1.5417
Epoch 9 of 30 : 969 out of 10000 = 0.0969; cost=1.4061
Epoch 10 of 30 : 979 out of 10000 = 0.0979; cost=1.309
Epoch 11 of 30 : 974 out of 10000 = 0.0974; cost=1.2383
Epoch 12 of 30 : 975 out of 10000 = 0.0975; cost=1.1794
Epoch 13 of 30 : 988 out of 10000 = 0.0988; cost=1.1257
Epoch 14 of 30 : 990 out of 10000 = 0.099; cost=1.0758
Epoch 15 of 30 : 1001 out of 10000 = 0.1001; cost=1.03
Epoch 16 of 30 : 997 out of 10000 = 0.0997; cost=0.9881
Epoch 17 of 30 : 992 out of 10000 = 0.0992; cost=0.9481
Epoch 18 of 30 : 977 out of 10000 = 0.0977; cost=0.9062
Epoc

In [139]:
# TRAIN using mini-batch gradient descent

# Initialize network
layers = [784, 30, 10]
weights = initializeWeights(layers)

# Take backup of weights to be used later for comparison
initialWeights = [np.array(w) for w in weights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 3.0

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate, testX=x_test, testY=y_test)

Epoch 1 of 30 : 8779 out of 10000 = 0.8779; cost=0.0698
Epoch 2 of 30 : 8828 out of 10000 = 0.8828; cost=0.0665
Epoch 3 of 30 : 9095 out of 10000 = 0.9095; cost=0.0545
Epoch 4 of 30 : 9098 out of 10000 = 0.9098; cost=0.0535
Epoch 5 of 30 : 9160 out of 10000 = 0.916; cost=0.0503
Epoch 6 of 30 : 9171 out of 10000 = 0.9171; cost=0.0496
Epoch 7 of 30 : 9245 out of 10000 = 0.9245; cost=0.0472
Epoch 8 of 30 : 9259 out of 10000 = 0.9259; cost=0.0484
Epoch 9 of 30 : 9224 out of 10000 = 0.9224; cost=0.0472
Epoch 10 of 30 : 9243 out of 10000 = 0.9243; cost=0.046
Epoch 11 of 30 : 9288 out of 10000 = 0.9288; cost=0.0443
Epoch 12 of 30 : 9259 out of 10000 = 0.9259; cost=0.0458
Epoch 13 of 30 : 9281 out of 10000 = 0.9281; cost=0.045
Epoch 14 of 30 : 9248 out of 10000 = 0.9248; cost=0.0474
Epoch 15 of 30 : 9239 out of 10000 = 0.9239; cost=0.0457
Epoch 16 of 30 : 9305 out of 10000 = 0.9305; cost=0.0449
Epoch 17 of 30 : 9290 out of 10000 = 0.929; cost=0.0449
Epoch 18 of 30 : 9296 out of 10000 = 0.9296;

We can see that sometimes the accuracy decreases and cost increases. This means our learning rate is too big to move only so much as to decrease the cost. We need to find ways of adapting the learning rate to train our network with the optimal weights.

# Optimizers for gradient descent

# 1. Decaying learning rate

By decreasing the learning rate, we will be able to take smaller steps towards the optimum. There are different ways to decrease the learning rate.

## Step decay

Whenever the cost of the neural network increases (i.e. accuracy decreases), we shall revert the weights back to the values at the start of the iteration, and reduce the learning rate by half. We call this **Step decay**.

In our training function, let us add the option of choosing the decay, and define the Step decay optimizer. If the cost does not decrease contiguously for too long (5 epochs), let us stop our training there.

In [221]:
# TRAINING USING MINI-BATCH GRADIENT DESCENT
def trainUsingMinibatchGD(weights, X, Y, minibatchSize, nEpochs, learningRate=1.0, 
                          decay=None, testX=None, testY=None):
    # If testing data is not provided, check accuracy on training data
    if testX is None:
        testX = X
        testY = Y
    
    # Check cost and accuracy
    # Initialize cost
    prevCost = nnCost(weights, testX, testY)
    yes = evaluate(weights, testX, testY)
    print("Before training: "+str(yes)+" out of "+str(len(testY))+" = "+str(round(float(yes/len(testY)),4))+
          "; cost="+str(prevCost))
    
    # Backup weights to revert back in case cost increases
    oldWeights = [np.array(w) for w in weights]
    
    # To count the number of times learning rate had to be halved contiguously
    countLRHalf = 0
    
    # Initialize index for iteration through epochs
    epoch = 0
    
    # For nEpochs number of epochs:
    while epoch < nEpochs:
        # clear output
        #clear_output()
        
        # Make a list of all the indices
        fullIdx = list(range(len(Y)))
        
        # Shuffle the full index
        np.random.shuffle(fullIdx)
        
        # Count number of mini-batches
        nOfMinibatches = int(len(X)/minibatchSize)
        
        # For each mini-batch
        for m in range(nOfMinibatches):
            
            # Compute the starting index of this mini-batch
            startIdx = m*minibatchSize
            
            # Declare sampled inputs and outputs
            xSample = X[fullIdx[startIdx:startIdx+minibatchSize]]
            ySample = Y[fullIdx[startIdx:startIdx+minibatchSize]]

            # Run backprop, with an optimizer
            backProp(weights, xSample, ySample, learningRate)
        
        # Check cost and accuracy
        cost = nnCost(weights, testX, testY)
        yes = evaluate(weights, testX, testY)
        print("Epoch "+str(epoch+1)+" of "+str(nEpochs)+" : "+
              str(yes)+" out of "+str(len(testY))+" = "+str(round(float(yes/len(testY)),4))+
              "; cost="+str(cost))
        
        # If decay type is 'step', when cost increases, revert back weights and halve learning rate 
        if decay is 'step':
            # If cost does not decrease
            if cost >= prevCost:
                # Revert weights back to those at the start of this epoch
                weights = [np.array(w) for w in oldWeights]
                
                # Recalculate prevCost
                cost = nnCost(weights, testX, testY)
                
                # Halve the learning rate
                learningRate = learningRate/2.0
                
                # Revert iteration number
                epoch -= 1
                
                # Increment the count of halving learning rate by 1
                countLRHalf += 1
                
                print("Halving learning rate to: "+str(learningRate)+", count="+str(countLRHalf))
            # If cost decreases, reset the count to 0
            else:
                countLRHalf = 0
        
        # If learningRate has been halved contiguously for too long, break
        if countLRHalf is 5:
            break
        
        # Set prevCost for next epoch
        prevCost = cost
        
        # Set oldWeights for next epoch
        oldWeights = [np.array(w) for w in weights]
        
        # Increase iteration number for epochs
        epoch += 1
    
    # If training was stopped because accuracy was not increasing
    if epoch < nEpochs:
        print("Training ended prematurely...")
    # If training ended in correct number of epochs
    else:
        print("Training complete.")

Now train the network with a decaying learning rate.

In [209]:
# TRAIN with decaying learning rate

# Revert to old weights
weights = [np.array(w) for w in initialWeights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 3.0

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate,
                      decay='step', testX=x_test, testY=y_test)

Before training: 1004 out of 10000 = 0.1004; cost=2.3706
Epoch 1 of 30 : 8903 out of 10000 = 0.8903; cost=0.0675467370522
Epoch 2 of 30 : 9009 out of 10000 = 0.9009; cost=0.0604533829713
Epoch 3 of 30 : 9105 out of 10000 = 0.9105; cost=0.0558005001716
Epoch 4 of 30 : 9185 out of 10000 = 0.9185; cost=0.050210922763
Epoch 5 of 30 : 9203 out of 10000 = 0.9203; cost=0.0501670595612
Epoch 6 of 30 : 9201 out of 10000 = 0.9201; cost=0.0469989425479
Epoch 7 of 30 : 9269 out of 10000 = 0.9269; cost=0.0442005669555
Epoch 8 of 30 : 9215 out of 10000 = 0.9215; cost=0.0476086394864
Halving learning rate to: 1.5, count=1
Epoch 8 of 30 : 9288 out of 10000 = 0.9288; cost=0.0440865277318
Epoch 9 of 30 : 9274 out of 10000 = 0.9274; cost=0.0433871957187
Epoch 10 of 30 : 9301 out of 10000 = 0.9301; cost=0.0436701848444
Halving learning rate to: 0.75, count=1
Epoch 10 of 30 : 9283 out of 10000 = 0.9283; cost=0.0425153081087
Epoch 11 of 30 : 9308 out of 10000 = 0.9308; cost=0.0423972816577
Epoch 12 of 30 : 

We see that by halving learning rate whenever cost increases, the final accuracy improves for the same number of epochs and same weight initialization.

## Time decay



# 2. Momentum update

Another way to improve accuracy faster is to push the change in weight more in the direction of the previous update. From [CS231n](http://cs231n.github.io/neural-networks-3/),
```
v = mu * v - learning_rate * dx # integrate velocity
x += v # integrate position
```

So this is an improvement on the back-propagation algorithm itself.

Let us add an option of choosing an optimization on back-propagation, and code Momentum:

In [222]:
# IMPLEMENTING BACK-PROPAGATION WITH LEARNING RATE, MOMENTUM
# Added eta, the learning rate, as an input
def backProp(weights, X, Y, learningRate, optimizer=None, mu=0.9):
    # Forward propagate to find outputs
    outputs = forwardProp(X, weights)
    
    # For the last layer, bpError = error = yPred - Y
    bpError = outputs[-1] - Y
    
    # Initialize velocity in the shape of weights for use with momentum
    v = [np.zeros(w.shape) for w in weights]
    
    # Back-propagating from the last layer to the first
    for l, w in enumerate(reversed(weights)):
        
        # Find yPred for this layer
        yPred = outputs[-l-1]
        
        # Calculate delta for this layer using bpError from next layer
        delta = np.multiply(np.multiply(bpError, yPred), 1-yPred)
        
        # Find input to the layer, by adding bias to the output of the previous layer
        # Take care, l goes from 0 to 1, while the weights are in reverse order
        if l==len(weights)-1: # If 1st layer has been reached
            xL = addBiasTerms(X)
        else:
            xL = addBiasTerms(outputs[-l-2])
        
        # Calculate the gradient for this layer
        grad = np.dot(delta.T, xL)/len(Y)
        
        # Calculate bpError for previous layer to be back-propagated
        bpError = np.dot(delta, w)
        
        # Ignore bias term in bpError
        bpError = bpError[:,1:]
        
        # Change weights of the current layer (W <- W + eta*deltaW)
        if optimizer is None:
            w += -learningRate * grad
        
        # Momentum
        if optimizer is 'momentum':
            v[-l-1] = mu * v[-l-1] - learningRate * grad
            w += v[-l-1]


Add the option of optimizer while calling backprop in trainUsingMiniGD:

In [223]:
# TRAINING USING MINI-BATCH GRADIENT DESCENT
def trainUsingMinibatchGD(weights, X, Y, minibatchSize, nEpochs, learningRate=1.0, 
                          decay=None, optimizer=None, mu=0.9, testX=None, testY=None):
    # If testing data is not provided, check accuracy on training data
    if testX is None:
        testX = X
        testY = Y
    
    # Check cost and accuracy
    # Initialize cost
    prevCost = nnCost(weights, testX, testY)
    yes = evaluate(weights, testX, testY)
    print("Before training: "+str(yes)+" out of "+str(len(testY))+" = "+str(round(float(yes/len(testY)),4))+
          "; cost="+str(prevCost))
    
    # Backup weights to revert back in case cost increases
    oldWeights = [np.array(w) for w in weights]
    
    # To count the number of times learning rate had to be halved contiguously
    countLRHalf = 0
    
    # Initialize index for iteration through epochs
    epoch = 0
    
    # For nEpochs number of epochs:
    while epoch < nEpochs:
        # clear output
        #clear_output()
        
        # Make a list of all the indices
        fullIdx = list(range(len(Y)))
        
        # Shuffle the full index
        np.random.shuffle(fullIdx)
        
        # Count number of mini-batches
        nOfMinibatches = int(len(X)/minibatchSize)
        
        # For each mini-batch
        for m in range(nOfMinibatches):
            
            # Compute the starting index of this mini-batch
            startIdx = m*minibatchSize
            
            # Declare sampled inputs and outputs
            xSample = X[fullIdx[startIdx:startIdx+minibatchSize]]
            ySample = Y[fullIdx[startIdx:startIdx+minibatchSize]]

            # Run backprop, with an optimizer
            backProp(weights, xSample, ySample, learningRate, optimizer, mu)
        
        # Check cost and accuracy
        cost = nnCost(weights, testX, testY)
        yes = evaluate(weights, testX, testY)
        print("Epoch "+str(epoch+1)+" of "+str(nEpochs)+" : "+
              str(yes)+" out of "+str(len(testY))+" = "+str(round(float(yes/len(testY)),4))+
              "; cost="+str(cost))
        
        # If decay type is 'step', when cost increases, revert back weights and halve learning rate 
        if decay is 'step':
            # If cost does not decrease
            if cost >= prevCost:
                # Revert weights back to those at the start of this epoch
                weights = [np.array(w) for w in oldWeights]
                
                # Recalculate prevCost
                cost = nnCost(weights, testX, testY)
                
                # Halve the learning rate
                learningRate = learningRate/2.0
                
                # Revert iteration number
                epoch -= 1
                
                # Increment the count of halving learning rate by 1
                countLRHalf += 1
                
                print("Halving learning rate to: "+str(learningRate)+", count="+str(countLRHalf))
            # If cost decreases, reset the count to 0
            else:
                countLRHalf = 0
        
        # If learningRate has been halved contiguously for too long, break
        if countLRHalf is 5:
            break
        
        # Set prevCost for next epoch
        prevCost = cost
        
        # Set oldWeights for next epoch
        oldWeights = [np.array(w) for w in weights]
        
        # Increase iteration number for epochs
        epoch += 1
    
    # If training was stopped because accuracy was not increasing
    if epoch < nEpochs:
        print("Training ended prematurely...")
    # If training ended in correct number of epochs
    else:
        print("Training complete.")

Train the network with the same initial weights, using step decay in learning rate, and Momentum.

In [224]:
# TRAIN using Momentum

# Revert to old weights
weights = [np.array(w) for w in initialWeights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 3.0
mu = 0.9

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate,
                      decay='step', optimizer='momentum', mu=mu, testX=x_test, testY=y_test)

Before training: 1004 out of 10000 = 0.1004; cost=2.37056621443
Epoch 1 of 30 : 8777 out of 10000 = 0.8777; cost=0.0760498120193
Epoch 2 of 30 : 8928 out of 10000 = 0.8928; cost=0.0603590744736
Epoch 3 of 30 : 9118 out of 10000 = 0.9118; cost=0.0527367063794
Epoch 4 of 30 : 9098 out of 10000 = 0.9098; cost=0.0519137464361
Epoch 5 of 30 : 9205 out of 10000 = 0.9205; cost=0.0500469055895
Epoch 6 of 30 : 9154 out of 10000 = 0.9154; cost=0.049017530374
Epoch 7 of 30 : 9197 out of 10000 = 0.9197; cost=0.0494850178497
Halving learning rate to: 1.5, count=1
Epoch 7 of 30 : 9301 out of 10000 = 0.9301; cost=0.0442050439134
Epoch 8 of 30 : 9298 out of 10000 = 0.9298; cost=0.0441667726679
Epoch 9 of 30 : 9299 out of 10000 = 0.9299; cost=0.0437103775144
Epoch 10 of 30 : 9285 out of 10000 = 0.9285; cost=0.0438829175281
Halving learning rate to: 0.75, count=1
Epoch 10 of 30 : 9331 out of 10000 = 0.9331; cost=0.0422593709095
Epoch 11 of 30 : 9306 out of 10000 = 0.9306; cost=0.0422729755298
Halving le

Better accuracy than vanilla gradient descent!

# 3. Nesterov Momentum

From [CS231n](http://cs231n.github.io/neural-networks-3/),
```
v_prev = v # back this up
v = mu * v - learning_rate * dx # velocity update stays the same
x += -mu * v_prev + (1 + mu) * v # position update changes form
```

In [225]:
# IMPLEMENTING BACK-PROPAGATION WITH LEARNING RATE, MOMENTUM, NAG
def backProp(weights, X, Y, learningRate, optimizer=None, mu=0.9):
    # Forward propagate to find outputs
    outputs = forwardProp(X, weights)
    
    # For the last layer, bpError = error = yPred - Y
    bpError = outputs[-1] - Y
    
    # Initialize velocity in the shape of weights for use with momentum and NAG
    v = [np.zeros(w.shape) for w in weights]
    prevV = [np.zeros(w.shape) for w in weights]
    
    # Back-propagating from the last layer to the first
    for l, w in enumerate(reversed(weights)):
        
        # Find yPred for this layer
        yPred = outputs[-l-1]
        
        # Calculate delta for this layer using bpError from next layer
        delta = np.multiply(np.multiply(bpError, yPred), 1-yPred)
        
        # Find input to the layer, by adding bias to the output of the previous layer
        # Take care, l goes from 0 to 1, while the weights are in reverse order
        if l==len(weights)-1: # If 1st layer has been reached
            xL = addBiasTerms(X)
        else:
            xL = addBiasTerms(outputs[-l-2])
        
        # Calculate the gradient for this layer
        grad = np.dot(delta.T, xL)/len(Y)
        
        # Calculate bpError for previous layer to be back-propagated
        bpError = np.dot(delta, w)
        
        # Ignore bias term in bpError
        bpError = bpError[:,1:]
        
        # CHANGE WEIGHTS of the current layer (W <- W + eta*deltaW)
        if optimizer is None:
            w += -learningRate * grad
        
        # Momentum
        if optimizer is 'momentum':
            v[-l-1] = mu * v[-l-1] - learningRate * grad
            w += v[-l-1]
        
        # Nesterov Momentum
        if optimizer is 'nag':
            prevV[-l-1] = np.array(v[-l-1]) # back this up
            v[-l-1] = mu * v[-l-1] - learningRate * grad # velocity update stays the same
            w += -mu * prevV[-l-1] + (1 + mu) * v[-l-1] # position update changes form


Train the network with the same initial weights, using step decay in learning rate, and Momentum.

In [226]:
# TRAIN using Nesterov Momentum (NAG)

# Revert to old weights
weights = [np.array(w) for w in initialWeights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 3.0
mu = 0.9

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate,
                      decay='step', optimizer='nag', mu=mu, testX=x_test, testY=y_test)

Before training: 1004 out of 10000 = 0.1004; cost=2.37056621443
Epoch 1 of 30 : 8849 out of 10000 = 0.8849; cost=0.0717524192533
Epoch 2 of 30 : 9091 out of 10000 = 0.9091; cost=0.055670681835
Epoch 3 of 30 : 9153 out of 10000 = 0.9153; cost=0.0525418892571
Epoch 4 of 30 : 9117 out of 10000 = 0.9117; cost=0.0535916565845
Halving learning rate to: 1.5, count=1
Epoch 4 of 30 : 9233 out of 10000 = 0.9233; cost=0.0481753882615
Epoch 5 of 30 : 9237 out of 10000 = 0.9237; cost=0.0457395937391
Epoch 6 of 30 : 9285 out of 10000 = 0.9285; cost=0.0452398507075
Epoch 7 of 30 : 9251 out of 10000 = 0.9251; cost=0.0457638468201
Halving learning rate to: 0.75, count=1
Epoch 7 of 30 : 9295 out of 10000 = 0.9295; cost=0.0440445403005
Epoch 8 of 30 : 9282 out of 10000 = 0.9282; cost=0.0428945881307
Epoch 9 of 30 : 9302 out of 10000 = 0.9302; cost=0.0433700328082
Halving learning rate to: 0.375, count=1
Epoch 9 of 30 : 9307 out of 10000 = 0.9307; cost=0.0428650912142
Epoch 10 of 30 : 9304 out of 10000 = 

# 4. Adagrad

From [CS231n](http://cs231n.github.io/neural-networks-3/),

```
# Assume the gradient dx and parameter vector x
cache += dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)
```

In [227]:
# IMPLEMENTING BACK-PROPAGATION WITH LEARNING RATE, MOMENTUM, NAG, ADAGRAD
def backProp(weights, X, Y, learningRate, optimizer=None, mu=0.9):
    # Forward propagate to find outputs
    outputs = forwardProp(X, weights)
    
    # For the last layer, bpError = error = yPred - Y
    bpError = outputs[-1] - Y
    
    # Initialize velocity in the shape of weights for use with momentum and NAG
    v = [np.zeros(w.shape) for w in weights]
    prevV = [np.zeros(w.shape) for w in weights]
    
    # Initialize cache for use with Adagrad
    cache = [np.zeros(w.shape) for w in weights]
    
    # Back-propagating from the last layer to the first
    for l, w in enumerate(reversed(weights)):
        
        # Find yPred for this layer
        yPred = outputs[-l-1]
        
        # Calculate delta for this layer using bpError from next layer
        delta = np.multiply(np.multiply(bpError, yPred), 1-yPred)
        
        # Find input to the layer, by adding bias to the output of the previous layer
        # Take care, l goes from 0 to 1, while the weights are in reverse order
        if l==len(weights)-1: # If 1st layer has been reached
            xL = addBiasTerms(X)
        else:
            xL = addBiasTerms(outputs[-l-2])
        
        # Calculate the gradient for this layer
        grad = np.dot(delta.T, xL)/len(Y)
        
        # Calculate bpError for previous layer to be back-propagated
        bpError = np.dot(delta, w)
        
        # Ignore bias term in bpError
        bpError = bpError[:,1:]
        
        # CHANGE WEIGHTS of the current layer (W <- W + eta*deltaW)
        if optimizer is None:
            w += -learningRate * grad
        
        # Momentum
        if optimizer is 'momentum':
            v[-l-1] = mu * v[-l-1] - learningRate * grad
            w += v[-l-1]
        
        # Nesterov Momentum
        if optimizer is 'nag':
            prevV[-l-1] = np.array(v[-l-1]) # back this up
            v[-l-1] = mu * v[-l-1] - learningRate * grad # velocity update stays the same
            w += -mu * prevV[-l-1] + (1 + mu) * v[-l-1] # position update changes form
        
        # Adagrad
        if optimizer is 'adagrad':
            cache[-l-1] += grad**2
            w += - learningRate * grad / (np.sqrt(cache[-l-1]) + np.finfo(float).eps)

In [228]:
# TRAIN using Adagrad

# Revert to old weights
weights = [np.array(w) for w in initialWeights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 3.0

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate,
                      decay='step', optimizer='adagrad', testX=x_test, testY=y_test)

Before training: 1004 out of 10000 = 0.1004; cost=2.37056621443


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Epoch 1 of 30 : 0 out of 10000 = 0.0; cost=0.5
Epoch 2 of 30 : 0 out of 10000 = 0.0; cost=0.5
Halving learning rate to: 1.5, count=1


KeyboardInterrupt: 

Looks like the learningRate is too high at the start.

In [229]:
# TRAIN using Adagrad

# Revert to old weights
weights = [np.array(w) for w in initialWeights]

# Set options of mini-batch gradient descent
minibatchSize = 10
nEpochs = 30
learningRate = 1.0

# Train
trainUsingMinibatchGD(weights, x_train, y_train, minibatchSize, nEpochs, learningRate,
                      decay='step', optimizer='adagrad', testX=x_test, testY=y_test)

Before training: 1004 out of 10000 = 0.1004; cost=2.37056621443


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Epoch 1 of 30 : 5806 out of 10000 = 0.5806; cost=0.260757491593
Epoch 2 of 30 : 6782 out of 10000 = 0.6782; cost=0.204815630246
Epoch 3 of 30 : 6842 out of 10000 = 0.6842; cost=0.192322763729
Epoch 4 of 30 : 6874 out of 10000 = 0.6874; cost=0.187409859878
Epoch 5 of 30 : 6444 out of 10000 = 0.6444; cost=0.206949182702
Halving learning rate to: 0.5, count=1
Epoch 5 of 30 : 6822 out of 10000 = 0.6822; cost=0.185802116025
Epoch 6 of 30 : 6915 out of 10000 = 0.6915; cost=0.185329131066
Epoch 7 of 30 : 7694 out of 10000 = 0.7694; cost=0.156780001919
Epoch 8 of 30 : 7686 out of 10000 = 0.7686; cost=0.147885292916
Epoch 9 of 30 : 7745 out of 10000 = 0.7745; cost=0.149509382291
Halving learning rate to: 0.25, count=1
Epoch 9 of 30 : 7776 out of 10000 = 0.7776; cost=0.142723029862
Epoch 10 of 30 : 7794 out of 10000 = 0.7794; cost=0.140961464727
Epoch 11 of 30 : 7650 out of 10000 = 0.765; cost=0.148943247373
Halving learning rate to: 0.125, count=1
Epoch 11 of 30 : 7715 out of 10000 = 0.7715; co